In [ ]:
pip install pyspark


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=0b232672cd2187fea44353e917f421bc6a0865e0a2f3aa4b02538a2d2599c91b
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("create_df_with_orc").getOrCreate()

In [ ]:
simpleData = [("James","Sales","NY",90000,34,10000),
    ("Michael","Sales","NY",86000,56,20000),
    ("Robert","Sales","CA",81000,30,23000),
    ("Maria","Finance","CA",90000,24,23000),
    ("Raman","Finance","CA",99000,40,24000),
    ("Scott","Finance","NY",83000,36,19000),
    ("Jen","Finance","NY",79000,53,15000),
    ("Jeff","Marketing","CA",80000,25,18000),
    ("Kumar","Marketing","NY",91000,50,21000)
  ]
schema = ["employee_name","department","state","salary","age","bonus"]
simple_df = spark.createDataFrame(simpleData,  schema)
simple_df.show()

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|        James|     Sales|   NY| 90000| 34|10000|
|      Michael|     Sales|   NY| 86000| 56|20000|
|       Robert|     Sales|   CA| 81000| 30|23000|
|        Maria|   Finance|   CA| 90000| 24|23000|
|        Raman|   Finance|   CA| 99000| 40|24000|
|        Scott|   Finance|   NY| 83000| 36|19000|
|          Jen|   Finance|   NY| 79000| 53|15000|
|         Jeff| Marketing|   CA| 80000| 25|18000|
|        Kumar| Marketing|   NY| 91000| 50|21000|
+-------------+----------+-----+------+---+-----+



In [ ]:
from pyspark.sql.functions import sum
from pyspark.sql.functions import col
from pyspark.sql import Window  #Importing the windowfunction
windowSpec  = Window.partitionBy("Department") #Partitioning by the department column
simple_df.withColumn("Salary Sum",sum(col("Salary")).over(windowSpec)).show() #Creating new column salary sum with sum function using over to calculate the sum on the dataframe with the partitioned rows

+-------------+----------+-----+------+---+-----+----------+
|employee_name|department|state|salary|age|bonus|Salary Sum|
+-------------+----------+-----+------+---+-----+----------+
|        Maria|   Finance|   CA| 90000| 24|23000|    351000|
|        Raman|   Finance|   CA| 99000| 40|24000|    351000|
|        Scott|   Finance|   NY| 83000| 36|19000|    351000|
|          Jen|   Finance|   NY| 79000| 53|15000|    351000|
|         Jeff| Marketing|   CA| 80000| 25|18000|    171000|
|        Kumar| Marketing|   NY| 91000| 50|21000|    171000|
|        James|     Sales|   NY| 90000| 34|10000|    257000|
|      Michael|     Sales|   NY| 86000| 56|20000|    257000|
|       Robert|     Sales|   CA| 81000| 30|23000|    257000|
+-------------+----------+-----+------+---+-----+----------+



In [ ]:
empSalary = simple_df.groupBy('department','state').agg({'salary':'sum','bonus':'sum'})
empSalary.withColumn('Total Sum',col('sum(bonus)')+col('sum(salary)')).show()

+----------+-----+----------+-----------+---------+
|department|state|sum(bonus)|sum(salary)|Total Sum|
+----------+-----+----------+-----------+---------+
|     Sales|   CA|     23000|      81000|   104000|
|   Finance|   CA|     47000|     189000|   236000|
|     Sales|   NY|     30000|     176000|   206000|
|   Finance|   NY|     34000|     162000|   196000|
| Marketing|   NY|     21000|      91000|   112000|
| Marketing|   CA|     18000|      80000|    98000|
+----------+-----+----------+-----------+---------+



In [ ]:
from pyspark.sql.functions import sum, avg, min, max

final_df = simple_df.groupby("Department").agg(sum("Salary").alias("TotalSalary"), avg("Salary").alias("AverageSalary"), min("Salary").alias("MinimumSalary"), max("Salary").alias("MaximumSalary"))

In [ ]:
final_df.show()

+----------+-----------+-----------------+-------------+-------------+
|Department|TotalSalary|    AverageSalary|MinimumSalary|MaximumSalary|
+----------+-----------+-----------------+-------------+-------------+
|     Sales|     257000|85666.66666666667|        81000|        90000|
|   Finance|     351000|          87750.0|        79000|        99000|
| Marketing|     171000|          85500.0|        80000|        91000|
+----------+-----------+-----------------+-------------+-------------+



In [ ]:
#get data from the link and add url
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
from pyspark import SparkFiles
spark.sparkContext.addFile(url)
#read file and show it schema
onlinedata_df = spark.read.csv("file://"+SparkFiles.get("iris.data"), header=True, inferSchema= True)
onlinedata_df.printSchema()
onlinedata_df.show()

root
 |-- 5.1: double (nullable = true)
 |-- 3.5: double (nullable = true)
 |-- 1.4: double (nullable = true)
 |-- 0.2: double (nullable = true)
 |-- Iris-setosa: string (nullable = true)

+---+---+---+---+-----------+
|5.1|3.5|1.4|0.2|Iris-setosa|
+---+---+---+---+-----------+
|4.9|3.0|1.4|0.2|Iris-setosa|
|4.7|3.2|1.3|0.2|Iris-setosa|
|4.6|3.1|1.5|0.2|Iris-setosa|
|5.0|3.6|1.4|0.2|Iris-setosa|
|5.4|3.9|1.7|0.4|Iris-setosa|
|4.6|3.4|1.4|0.3|Iris-setosa|
|5.0|3.4|1.5|0.2|Iris-setosa|
|4.4|2.9|1.4|0.2|Iris-setosa|
|4.9|3.1|1.5|0.1|Iris-setosa|
|5.4|3.7|1.5|0.2|Iris-setosa|
|4.8|3.4|1.6|0.2|Iris-setosa|
|4.8|3.0|1.4|0.1|Iris-setosa|
|4.3|3.0|1.1|0.1|Iris-setosa|
|5.8|4.0|1.2|0.2|Iris-setosa|
|5.7|4.4|1.5|0.4|Iris-setosa|
|5.4|3.9|1.3|0.4|Iris-setosa|
|5.1|3.5|1.4|0.3|Iris-setosa|
|5.7|3.8|1.7|0.3|Iris-setosa|
|5.1|3.8|1.5|0.3|Iris-setosa|
|5.4|3.4|1.7|0.2|Iris-setosa|
+---+---+---+---+-----------+
only showing top 20 rows



In [ ]:
#Renamed Column Name using withColumnRenamed function
new_onlinedata_df = onlinedata_df.withColumnRenamed('5.1', 'sepal_length') \
                    .withColumnRenamed('3.5', 'sepal_width') \
                    .withColumnRenamed('1.4', 'petal_length') \
                    .withColumnRenamed('0.2', 'petal_width') \
                    .withColumnRenamed('Iris-setosa', 'species')
new_onlinedata_df.show()              

+------------+-----------+------------+-----------+-----------+
|sepal_length|sepal_width|petal_length|petal_width|    species|
+------------+-----------+------------+-----------+-----------+
|         4.9|        3.0|         1.4|        0.2|Iris-setosa|
|         4.7|        3.2|         1.3|        0.2|Iris-setosa|
|         4.6|        3.1|         1.5|        0.2|Iris-setosa|
|         5.0|        3.6|         1.4|        0.2|Iris-setosa|
|         5.4|        3.9|         1.7|        0.4|Iris-setosa|
|         4.6|        3.4|         1.4|        0.3|Iris-setosa|
|         5.0|        3.4|         1.5|        0.2|Iris-setosa|
|         4.4|        2.9|         1.4|        0.2|Iris-setosa|
|         4.9|        3.1|         1.5|        0.1|Iris-setosa|
|         5.4|        3.7|         1.5|        0.2|Iris-setosa|
|         4.8|        3.4|         1.6|        0.2|Iris-setosa|
|         4.8|        3.0|         1.4|        0.1|Iris-setosa|
|         4.3|        3.0|         1.1| 

In [ ]:
from pyspark.sql.functions import greatest
df = new_onlinedata_df.withColumn("MaxValue", greatest(col("sepal_length"), col("petal_length")))
df.show()

+------------+-----------+------------+-----------+-----------+--------+
|sepal_length|sepal_width|petal_length|petal_width|    species|MaxValue|
+------------+-----------+------------+-----------+-----------+--------+
|         4.9|        3.0|         1.4|        0.2|Iris-setosa|     4.9|
|         4.7|        3.2|         1.3|        0.2|Iris-setosa|     4.7|
|         4.6|        3.1|         1.5|        0.2|Iris-setosa|     4.6|
|         5.0|        3.6|         1.4|        0.2|Iris-setosa|     5.0|
|         5.4|        3.9|         1.7|        0.4|Iris-setosa|     5.4|
|         4.6|        3.4|         1.4|        0.3|Iris-setosa|     4.6|
|         5.0|        3.4|         1.5|        0.2|Iris-setosa|     5.0|
|         4.4|        2.9|         1.4|        0.2|Iris-setosa|     4.4|
|         4.9|        3.1|         1.5|        0.1|Iris-setosa|     4.9|
|         5.4|        3.7|         1.5|        0.2|Iris-setosa|     5.4|
|         4.8|        3.4|         1.6|        0.2|

In [ ]:
from pyspark.sql.functions import greatest, col, array_max,array,lit
max_sepal_length = new_onlinedata_df.agg(max("sepal_length")).collect()[0][0]
max_sepal_width = new_onlinedata_df.agg(max("sepal_width")).collect()[0][0]
max_petal_length = new_onlinedata_df.agg(max("petal_length")).collect()[0][0]
max_petal_width = new_onlinedata_df.agg(max("petal_width")).collect()[0][0]
max_sepal_width

4.4

In [ ]:
if max_sepal_length > max_petal_length:
  print(f"Sepal length has highest length with {max_sepal_length}")
else:
  print(f"petal length has highest length with {max_petal_length}")

Sepal length has highest length with 7.9


In [ ]:
if max_sepal_width > max_petal_width:
  print(f"Sepal width has highest length with {max_sepal_width}")
else:
  print(f"petal width has highest length with {max_petal_width}")

Sepal width has highest length with 4.4


In [ ]:
data = [('James','','Smith','1991-04-01','M',3000),
  ('Michael','Rose','','2000-05-19','M',4000),
  ('Robert','','Williams','1978-09-05','M',4000),
  ('Maria','Anne','Jones','1967-12-01','F',4000),
  ('Jen','Mary','Brown','1980-02-17','F',-1)
]
Columns = ['F_name', 'M_name','L_name', "DOB","Gender", "salary"]
#Create Dataframe from given data and column
employee_df = spark.createDataFrame(data, Columns)
employee_df.show()

+-------+------+--------+----------+------+------+
| F_name|M_name|  L_name|       DOB|Gender|salary|
+-------+------+--------+----------+------+------+
|  James|      |   Smith|1991-04-01|     M|  3000|
|Michael|  Rose|        |2000-05-19|     M|  4000|
| Robert|      |Williams|1978-09-05|     M|  4000|
|  Maria|  Anne|   Jones|1967-12-01|     F|  4000|
|    Jen|  Mary|   Brown|1980-02-17|     F|    -1|
+-------+------+--------+----------+------+------+



In [ ]:
#Datatype of each column
employee_df.printSchema()

root
 |-- F_name: string (nullable = true)
 |-- M_name: string (nullable = true)
 |-- L_name: string (nullable = true)
 |-- DOB: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- salary: long (nullable = true)



In [ ]:
#Convert datatype of salary using integer
df = employee_df.withColumn("salary", col("salary").cast("integer"))
df.printSchema()

root
 |-- F_name: string (nullable = true)
 |-- M_name: string (nullable = true)
 |-- L_name: string (nullable = true)
 |-- DOB: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- salary: integer (nullable = true)



In [ ]:
employee_df_salary5 = employee_df.withColumn("Salary5", col("salary") * 5)
employee_df_salary5.show()

+-------+------+--------+----------+------+------+-------+
| F_name|M_name|  L_name|       DOB|Gender|salary|Salary5|
+-------+------+--------+----------+------+------+-------+
|  James|      |   Smith|1991-04-01|     M|  3000|  15000|
|Michael|  Rose|        |2000-05-19|     M|  4000|  20000|
| Robert|      |Williams|1978-09-05|     M|  4000|  20000|
|  Maria|  Anne|   Jones|1967-12-01|     F|  4000|  20000|
|    Jen|  Mary|   Brown|1980-02-17|     F|    -1|     -5|
+-------+------+--------+----------+------+------+-------+

